In [1]:
import json
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import tqdm

In [2]:
with open('labels/bdd100k_labels_images_train.json') as json_file:
    data = json.load(json_file)

In [3]:
print("Total images in the training dataset: ", len(data))

69863

In [4]:
for datapoint in tqdm.tqdm(data):
    image = plt.imread('images/100k/train/'+datapoint['name'])
    drivable_polygons = list()
    pedestrian_polygons = list()
    
    for label in datapoint['labels']:
        if label['category'] in ['person']:
            polygon = Polygon([(label['box2d']['x1'], label['box2d']['y1']),
                               (label['box2d']['x1'], label['box2d']['y2']),
                               (label['box2d']['x2'], label['box2d']['y2']),
                               (label['box2d']['x2'], label['box2d']['y1']),
                              ])
            pedestrian_polygons.append(polygon)
        if label['category'] in ['drivable area']:
            polygon = Polygon(label['poly2d'][0]['vertices'])
            drivable_polygons.append(polygon)
        
    intersects_drivable = False
    for pedestrian in pedestrian_polygons: 
        for drivable in drivable_polygons: 
            intersects_drivable = pedestrian.intersects(drivable) or intersects_drivable

    if intersects_drivable:
        !cp {'images/100k/train/'+datapoint['name']} crossing/unlabeled
        
        # untoggle this code to visualise the image and how it intersects
#         plt.figure()
#         plt.imshow(image)
        
#         for polygon1 in drivable_polygons: 
#             plt.plot(*polygon1.exterior.xy, c='red')
#         for polygon1 in pedestrian_polygons: 
#             plt.plot(*polygon1.exterior.xy, c='green')

#         plt.show()


100%|██████████| 69863/69863 [20:07<00:00, 57.87it/s]
